<a href="https://colab.research.google.com/github/cascadenite/inspector_app/blob/main/Mobile_copy_of_Training_Object_Detection_with_TensorFlow_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#    Training an Object Detection Model with TensorFlow API using Google COLAB

Using a fast RCNN model on just 2 classes got loss down to 0.1 and tested very well.  However that model is not suitable for mobile so this copy will try one of  the more recent mobile models.


> Follow this medium post for the tutorial. https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b


This one for how to mobile it:

https://towardsdatascience.com/real-time-mobile-video-object-detection-using-tensorflow-a75fa0c5859d

https://github.com/ivanliu1989/Real-time-Mobile-Video-Object-Detection


>  go to Runtime > Change Runtime Type and make sure to select GPU as Hardware accelerator.




# import and builds

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
1.15.2


In [3]:
import tensorflow as tf

In [4]:
print(tf.__version__)

1.15.2


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
%cd '/content/gdrive/My Drive/Inspector'


/content/gdrive/My Drive/Inspector


Install some needed tools and dependencies.

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

In [ ]:
%cd '/content/gdrive/My Drive/Inspector/models/research'
!protoc object_detection/protos/*.proto --python_out=.

In [9]:
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/Inspector/models/research/:/content/gdrive/My Drive/Inspector/models/research/slim'

Always run the cell below for every session restart

In [ ]:
!python setup.py build
!python setup.py install

Check the remaining time allowed for the session you are runing. Depending on your data try to know the number of hours that will be enough. If the time is lesser then restart the session. NB the time result you see in in hours

In [11]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.709801106055577


In [ ]:
%%bash

export PROJECT_DIR='/content/gdrive/My Drive/Inspector/models/research/object_detection'

# Install COCO API
cd $PROJECT_DIR/tf-models
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI
make
# If there is error, try to specify using python3 as below
python3 setup.py build_ext install

# Copy compiled modules
cp -r pycocotools $PROJECT_DIR/tf-models/research/

Run Code below to see if all we need for the training has been installed.

In [ ]:

%cd /content/gdrive/My Drive/Inspector/models/research/object_detection/builders/
!python model_builder_test.py

Change Directory to object detection folder

In [14]:
%cd /content/gdrive/My Drive/Inspector/models/research/object_detection/

/content/gdrive/My Drive/Inspector/models/research/object_detection


In [ ]:
#cp '/content/test.tfrecord' '/content/drive/My Drive/Inspector/models/research/object_detection/data/'

Get the pre-trained Object detection model from TensorFlow with the code below

In [15]:
!pwd

/content/gdrive/My Drive/Inspector/models/research/object_detection


In [16]:
#mv '/content/pipeline.config' '/mydrive/Inspector/models/research/object_detection/training'

In [17]:
#cp -r '/content/gdrive/My Drive/Inspector/Mymodels/object_detection' '/content/gdrive/My Drive/Inspector/models/research/object_detection'

ALWAYS CHECK TIME

In [18]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.683574268288083


In [ ]:
# Current model in use:

!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz
!tar -xvf ssd_mobilenet_v3_large_coco_2020_01_14.tar.gz

In [20]:


#!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
#!tar -xvf ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz

In [21]:
#!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
#!tar -xvf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [ ]:
!pwd

# Perform training

# Tensorboard

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [24]:
model_dir = 'training/'
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

In [25]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"#

https://38a00da269eb.ngrok.io


Ensure to run the code cell below first

In [26]:
%load_ext tensorboard

IF you want to keep track of your model loss it best to run tensor board in background just like this before you run your training

In [ ]:
%tensorboard --logdir training/

In [28]:
!ln -s "/content/gdrive/My Drive" /Thisdrive

In [29]:
cd '/Thisdrive/Inspector/models/research/object_detection/'

/content/gdrive/My Drive/Inspector/models/research/object_detection


In [30]:
!pwd

/content/gdrive/My Drive/Inspector/models/research/object_detection


In [31]:
cp '/content/gdrive/My Drive/Inspector/models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001' '/Thisdrive/Inspector/models/research/object_detection/training/'

cp: cannot stat '/content/gdrive/My Drive/Inspector/models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001': No such file or directory


In [32]:
PATH_TO_ANNOTATIONS_FOLDER = '/content/drive/My Drive/Inspector/models/research/object_detection/data'

In [ ]:
!python3 generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=data/test.tfrecord  --img_path=Images/test

Ensure to copy train.py file from Legacy folder in your google drive to the object detection folder itself before runnning the cell below

In [35]:
pipeline_config_path='/mydrive/Inspector/models/research/object_detection/pipeline.config'

# Launch the training!!

In [ ]:
#python model_main.py --alsologtostderr --model_dir=training/ --pipeline_config_path=training/ssd_inception_v2_coco.config

In [ ]:
!python model_main.py --alsologtostderr --model_dir='/content/gdrive/My Drive/Inspector/models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28/' --pipeline_config_path='/content/gdrive/My Drive/Inspector/models/research/object_detection/training/Newpipeline2.config'

In [ ]:
tf.train.list_variables('/content/gdrive/My Drive/Inspector/models/research/object_detection/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/')

#Exporting inference graph

In [ ]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path /Thisdrive/Inspector/models/research/object_detection/training/ssdv3pipeline.config --trained_checkpoint_prefix training/model.ckpt-182132 --output_directory trained_inference_graph/

##ZIPPing the EXported graph

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!zip -r ssdv3_180k_graph.zip trained_inference_graph

#LETS TEST OUR MODEL AND SEE RESULT

> Copy about 9 images in the test_images folder and rename each to imagesX.jpg with X being a number from 1 to 9 before running the code below.



In [37]:
cp '/content/gdrive/My Drive/Inspector/models/research/object_detection/Images/test/toSortIMG_1598.jpg' '/content/gdrive/My Drive/Inspector/models/research/object_detection/test_images/'

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util



### Model preparation variable
MODEL_NAME = 'trained_inference_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = '/content/gdrive/My Drive/Inspector/models/research/object_detection/training/label_mapTwo.pbtxt'
NUM_CLASSES = 2 #remember number of objects you are training? cool.


### Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


###Loading label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)




### Load image into numpy function
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)




###STATING THE PATH TO IMAGES TO BE TESTED
PATH_TO_TEST_IMAGES_DIR = 'test_images/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'images{}.jpg'.format(i)) for i in range(1, 14) ]
IMAGE_SIZE = (12, 8)




### Function to run inference on a single image which will later be used in an iteration
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.8), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict



### To iterate on each image in the test image path defined 
### NB define the range of numbers and let it match the number of imAGES IN TEST FOLDER +1
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=1)
  display(Image.fromarray(image_np))
  



# Results of testing the zipped model.

copied 9 of the test images  all ones with xml files so must have annotations.

The results were very good - predicted wallcracks and flaking paint (one error was saying Lola's shoes were flaking paint!)

Wallcrack was not as good as it idnetified a curved joint (that does look a bit like a crack but isn't) on the coving as a crack.

Did the following to get the above:
1. got rid of all woodDamage and chippedPaint images and annotations
2. removed images that have no objects
3. took 72 new photos of my house with just flakingpaint and wallcracks
4. annotated these new ones and merged with the old train folder
5. created new csv files.

having trained ssd v2 for 100k steps and getting to about a loss of 2.  Next is to try and and put it in an App.

# Make it Mobile

In [ ]:
!export CONFIG_FILE='/Thisdrive/Inspector/models/research/object_detection/training/ssdv3pipeline.config'

In [ ]:
!export CHECKPOINT_PATH='/content/gdrive/My Drive/Inspector/models/research/object_detection/training/model.ckpt-182132'

In [ ]:
!export OUTPUT_DIR=/tmp/tflite

In [ ]:
BAZEL_VERSION = '0.25.2'

In [ ]:
!wget https://github.com/bazelbuild/bazel/releases/download/{BAZEL_VERSION}/bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

In [ ]:
!chmod +x bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

In [ ]:
!./bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

In [ ]:
!sudo apt install python-dev python-pip

In [ ]:
#!git clone https://github.com/tensorflow/tensorflow.git
#%cd tensorflow

In [ ]:
#!python ./configure.py


In [ ]:
#%cd tensorflow

/content/gdrive/My Drive/Inspector/models/research/object_detection/tensorflow


In [ ]:
#!bazel build --config=v1 /Thisdrive/Inspector/models/research/object_detection/tensorflow/tools/pip_package:build_pip_package 

ERROR: bazel does not currently work properly from paths containing spaces (com.google.devtools.build.lib.runtime.BlazeWorkspace@3ab6523a).


In [ ]:
#!bazel build -c opt --config=android_arm{,64} --cxxopt='--std=c++11' \
//tensorflow/lite/examples/android:tflite_demo

In [ ]:
!python export_tflite_ssd_graph.py --pipeline_config_path='/Thisdrive/Inspector/models/research/object_detection/training/ssdv3pipeline.config' \
--trained_checkpoint_prefix='/content/gdrive/My Drive/Inspector/models/research/object_detection/training/model.ckpt-182132' \
--output_directory=/Thisdrive/Inspector/models/research/object_detection/tflite \
--add_postprocessing_op=true

In [ ]:
#%cd tensorflow/

In [ ]:
#!bazel run -c opt tensorflow/contrib/lite/toco:toco


In [ ]:
!toco \
 --graph_def_file='/Thisdrive/Inspector/models/research/object_detection/tflite/tflite_graph.pb' \
 --output_file='/Thisdrive/Inspector/models/research/object_detection/tflite/detectv3.tflite' \
 --input_shapes=1,300,300,3 \
 --input_arrays=normalized_input_image_tensor \
 --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
 --inference_type=FLOAT \
 --allow_custom_ops

https://medium.com/datadriveninvestor/how-to-train-your-own-custom-model-with-tensorflow-object-detection-api-and-deploy-it-into-android-aeacab7fa76f
